<a href="https://colab.research.google.com/github/fyuuki0jp/colab_llm_training/blob/main/gemma2_2b_jpn_it_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq transformers datasets bitsandbytes torch xformers torchinfo
!pip install -Uq unsloth
# 最新バージョンは割と変わるので注意する。このコミットID(bc5f726a3cba3dbacda604a288dbc352c0baa737)より新しいと学習できない
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps "git+https://github.com/unslothai/unsloth.git@bc5f726a3cba3dbacda604a288dbc352c0baa737"
!pip install -Uq unsloth-zoo
# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 445.3 kB/s eta 0:00:00
   ━━━

In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 8196  # モデルが処理できるシーケンスの最大長
dtype = None  # 適切なデータ型を自動的に検出
load_in_4bit = False  # 4ビット量子化でメモリ効率を向上

# モデルとトークナイザをロード
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "fyuuki0jp/gemma-2-2b-jpn-it-shortwindow",  # 使用するモデルを選択。
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2024.12.11: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/

Unsloth: fyuuki0jp/gemma-2-2b-jpn-it-shortwindow can only handle sequence lengths of at most 8192.
But with kaiokendev's RoPE scaling of 1.0, it can be magically be extended to 8196!


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

In [ ]:
# @title 学習パラメータの設定 (最終層のLinear層はGemma2の場合embedding層と同一パラメータを利用する形で実装されている)
from torchinfo import summary

# 学習しないパラメータの設定
for layer in model.model.layers:
  # mlp層の学習はやらない
  for param in layer.mlp.parameters():
    param.requires_grad = False
  # attension層はKVでattention matrixを作るベクトルだけファインチューニングする(イメージ的には文の注意する箇所を調整したい)
  for name,param in layer.self_attn.named_parameters():
    # KVだけファインチューニングする
    if 'k' in name or 'v' in name:
      param.requires_grad = True
    else:
      param.requires_grad = False

# 学習対象のパラメータのrequires_gradをTrueにする
for param in model.model.embed_tokens.parameters():
  param.requires_grad = True
for param in model.model.norm.parameters():
  param.requires_grad = True
for param in model.lm_head.parameters():
  param.requires_grad = True

model = FastLanguageModel.for_training(model)
summary(model)

Layer (type:depth-idx)                                                      Param #
Gemma2ForCausalLM                                                           --
├─Gemma2Model: 1-1                                                          --
│    └─Embedding: 2-1                                                       589,824,000
│    └─ModuleList: 2-2                                                      --
│    │    └─Gemma2DecoderLayer: 3-1                                         77,865,984
│    │    └─Gemma2DecoderLayer: 3-2                                         77,865,984
│    │    └─Gemma2DecoderLayer: 3-3                                         77,865,984
│    │    └─Gemma2DecoderLayer: 3-4                                         77,865,984
│    │    └─Gemma2DecoderLayer: 3-5                                         77,865,984
│    │    └─Gemma2DecoderLayer: 3-6                                         77,865,984
│    │    └─Gemma2DecoderLayer: 3-7                                  

In [ ]:
from datasets import load_dataset,interleave_datasets

In [ ]:
eng_cot_dataset = load_dataset("O1-OPEN/OpenO1-SFT", split="train")
def transform_eng_cot_dataset(row):

    row['text'] = f"<start_of_turn>system \n You are a helpful assistant and must respond appropriately to user requests.<end_of_turn>\n\n<start_of_turn>user\n{row['instruction']}<end_of_turn>\n<start_of_turn>model\n{row['output'].replace('<Thought>','<thinking>').replace('</Thought>','</thinking>')}<end_of_turn><eos>"
    row['language'] = 'english'
    return row
eng_cot_dataset_train = eng_cot_dataset.map(transform_eng_cot_dataset).select_columns(['text','language'])

jpn_cot_dataset = load_dataset("jaeyong2/ja-rag-cot", split="train")

def transform_jpn_cot_dataset(row):
    row['text'] = f"<start_of_turn>system \nあなたは親切なアシスタントです。ユーザーの質問に対して情報源を元に回答してください。<end_of_turn>\n<start_of_turn>user \n{row['Question']}<end_of_turn><start_of_content>{row['context']}<end_of_content>\n<start_of_turn>model \n<thinking>{row['Thinking']} </thinking> <output> {row['Final Answer']} </output> <end_of_turn><eos>"
    row['language'] = 'jpn'
    return row

jpn_cot_dataset_train = jpn_cot_dataset.map(transform_jpn_cot_dataset).select_columns(['text','language'])

dolly_cot_dataset1 = load_dataset("fyuuki0jp/databricks-dolly-15k-ja-cot-part1",split='train')
# dolly_cot_dataset2 = load_dataset("fyuuki0jp/databricks-dolly-15k-ja-cot-part2",split='train')

def transform_dolly_cot_dataset(row):
  row['text'] = f"<start_of_turn>system\nあなたは親切なアシスタントです。ユーザーの質問に対して情報源を元に回答してください。<end_of_turn>\n<start_of_turn>user\n{row['instruction']}<end_of_turn>\n<start_of_turn>model\n<thinking>{row['cot']}</thinking><output>{row['response']}</output><end_of_turn><eos>"
  row['language'] = 'jpn'
  return row

dolly_cot_dataset1_train = dolly_cot_dataset1.map(transform_dolly_cot_dataset).select_columns(['text','language'])
# dolly_cot_dataset2_train = dolly_cot_dataset2.map(transform_dolly_cot_dataset).select_columns(['text','language'])


README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

OpenO1-SFT.jsonl:   0%|          | 0.00/384M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/77685 [00:00<?, ? examples/s]

Map:   0%|          | 0/77685 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

...

Map:   0%|          | 0/209496 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/465 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5271 [00:00<?, ? examples/s]

Map:   0%|          | 0/5271 [00:00<?, ? examples/s]

In [ ]:
datasets = [jpn_cot_dataset_train,eng_cot_dataset_train,dolly_cot_dataset1_train]# dolly_cot_dataset2_train
total_rows = sum([ds.num_rows for ds in datasets])
prob = [float(ds.num_rows)/total_rows for ds in datasets]
prob

[0.716343194780682, 0.26563333470107914, 0.01802347051823889]

In [ ]:
train_dataset = interleave_datasets(
    datasets,
    probabilities=prob,
    seed=42
)
train_dataset

Dataset({
    features: ['text', 'language'],
    num_rows: 290703
})

In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported, UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,                    # トレーニング対象のモデル
    tokenizer = tokenizer,            # モデル用トークナイザー
    train_dataset = train_dataset,          # トレーニングデータセット
    dataset_text_field = "text",      # データセット内のテキストフィールド名
    max_seq_length = max_seq_length,  # 最大シーケンス長
    dataset_num_proc = 6,             # データセット処理に使用するプロセス数

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,      # 各デバイスごとのバッチサイズ
        gradient_accumulation_steps = 32,     # 勾配の累積ステップ数

        # 長時間のトレーニングに使用可能な設定
        #max_steps = 2000,                     # トレーニングの最大ステップ数
        warmup_steps = 10,                   # ウォームアップステップ数
        # warmup_ratio = 0.1,                # ウォームアップ比率（オプション）
        num_train_epochs = 1,              # トレーニングのエポック数（オプション）

        # 埋め込み行列には通常より2～10倍小さい学習率を選択
        learning_rate = 5e-5,                # 全体の学習率
        embedding_learning_rate = 1e-5,      # 埋め込み層の学習率

        fp16 = not is_bfloat16_supported(),  # FP16を使用（bfloat16がサポートされていない場合）
        bf16 = is_bfloat16_supported(),      # bfloat16を使用（サポートされている場合）
        logging_steps = 1,                   # ログを記録するステップ間隔
        optim = "adamw_8bit",                # 8bit版AdamWオプティマイザーを使用
        weight_decay = 0.01,                 # 重み減衰率
        lr_scheduler_type = "linear",        # 学習率スケジューラのタイプ
        seed = 3407,                         # 再現性のための乱数シード
        output_dir = "outputs",              # 出力ディレクトリ
        report_to = "none",                  # ログ出力先（例: "wandb"などを指定可能）
    ),
)

Map (num_proc=6):   0%|          | 0/290703 [00:00<?, ? examples/s]

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 290,703 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 32
\        /    Total batch size = 64 | Total steps = 4,542
 "-____-"     Number of trainable parameters = 712,509,696
AUTOTUNE bmm(16x1591x256, 16x256x1591)
  triton_bmm_14 0.4157 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_10 0.4639 ms 89.6% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_5 0.4813 ms 86.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_13 0.5007 ms 83.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BL

Step,Training Loss
1,3.027900
2,3.317000
3,3.129100
4,2.929800
5,2.920500
6,2.668000
7,2.635700
8,2.529400
9,2.376800
10,2.322500


AUTOTUNE bmm(16x8196x256, 16x256x8196)
  triton_bmm_162 9.5519 ms 100.0% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  triton_bmm_166 9.6082 ms 99.4% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=128, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=4, num_warps=8
  bmm 10.3148 ms 92.6% 
  triton_bmm_157 10.3485 ms 92.3% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=16, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, num_warps=4
  triton_bmm_161 11.0039 ms 86.8% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=128, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=3, num_warps=4
  triton_bmm_158 11.0213 ms 86.7% ACC_TYPE='tl.float32', ALLOW_TF32=False, BLOCK_K=32, BLOCK_M=64, BLOCK_N=64, B_PROLOGUE_CAST_TYPE=None, EVEN_K=False, GROUP_M=8, num_stages=2, nu

In [ ]:
new_model_id = "fyuuki0jp/gemma-2-2b-jpn-it"
model.push_to_hub_merged(
    new_model_id+"_finetuning_sft",
    tokenizer=tokenizer,
    private=True
)

Unsloth: You are pushing to hub, but you passed your HF username = fyuuki0jp.
We shall truncate fyuuki0jp/gemma-2-2b-jpn-it_finetuning_sft to gemma-2-2b-jpn-it_finetuning_sft
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 54.17 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 26/26 [00:00<00:00, 650.30it/s]


Unsloth: Saving tokenizer...

  0%|          | 0/2 [00:00<?, ?it/s]

...

README.md:   0%|          | 0.00/602 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/fyuuki0jp/gemma-2-2b-jpn-it_finetuning_sft
